# Quick start

- To check data and model if it preditcs or not.
- This is only for intent dataset
- Model implemented in for 8640 data samples after filtering

# 1)- Import Key Modules

In [1]:
# support both Python 2 and Python 3 with minimal overhead.
from __future__ import absolute_import, division, print_function

# I am an engineer. I care only about error not warning. So, let's be maverick and ignore warnings.
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pickle
import pandas as pd
import numpy as np
from nltk.stem.porter import PorterStemmer
import re
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

In [3]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup

In [4]:
import itertools
import os

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from sklearn.preprocessing import LabelBinarizer, LabelEncoder
from sklearn.metrics import confusion_matrix

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.preprocessing import text, sequence
from keras import utils

Using TensorFlow backend.


# 2)- Loading Data

In [5]:
data=pd.read_pickle('file_clean_intent.pkl')

In [7]:
data.dep.nunique()

30

In [8]:
data.dep.value_counts()[:10]

Order management                                        4766
product complaints - products (Reklamation Produkte)     816
Software/Webshop/App                                     776
Shipping issues                                          482
Customer feedback                                        461
ShareWithSaal                                            354
Payment (Bezahlung)                                      321
product complaints - colours (Reklamation Farben)        257
Product (Produkt)                                        252
Marketing                                                155
Name: dep, dtype: int64

In [9]:
counts=data['dep'].value_counts()
df = data.loc[data['dep'].isin(counts.index[counts > 150])]

In [10]:
print(data.shape)
print(df.shape)

(9477, 4)
(8640, 4)


In [11]:
df.dep.value_counts()

Order management                                        4766
product complaints - products (Reklamation Produkte)     816
Software/Webshop/App                                     776
Shipping issues                                          482
Customer feedback                                        461
ShareWithSaal                                            354
Payment (Bezahlung)                                      321
product complaints - colours (Reklamation Farben)        257
Product (Produkt)                                        252
Marketing                                                155
Name: dep, dtype: int64

In [12]:
df.dep.nunique()

10

In [13]:
myfeat=["Order management","product complaints - products (Reklamation Produkte)","Software/Webshop/App",
        "Shipping issues","Customer feedback","ShareWithSaal","Payment (Bezahlung)",
        "product complaints - colours (Reklamation Farben)","Product (Produkt)","Marketing"]

### Data Clean

In [14]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
df['clean'] = df['firstmessage'].apply(clean_text)

### Train-Test

In [15]:
train_size = int(len(df) * 0.70)
train_posts = df['clean'][:train_size]
train_tags = df['dep'][:train_size]

test_posts = df['clean'][train_size:]
test_tags = df['dep'][train_size:]

In [16]:
print(train_posts.shape)

(6048,)


### Tokenization

In [17]:
max_words = 1000
tokenize = text.Tokenizer(num_words=max_words,char_level=False)
tokenize.fit_on_texts(train_posts) # only fit on train

x_train = tokenize.texts_to_matrix(train_posts)
x_test = tokenize.texts_to_matrix(test_posts)

In [18]:
print(x_train.shape)

(6048, 1000)


In [19]:
x_train[:5]

array([[0., 0., 1., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### encoder

In [20]:
encoder = LabelEncoder()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags) 
y_test = encoder.transform(test_tags)

In [21]:
num_classes = np.max(y_train) + 1 #strings start from 0 index. To count them from 1-10
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)

In [22]:
print(y_train.shape)
print(y_test.shape)

(6048, 10)
(2592, 10)


### Model Building

In [23]:
batch_size = 32
epochs = 10
shape_of_input=max_words # 1000 words

In [34]:
model = Sequential()
model.add(Dense(512, input_shape=(shape_of_input,)))
model.add(Activation('relu'))
model.add(Dropout(0.2))#The dropout rate is set to 20%, meaning one in 5 inputs will be randomly excluded from each update cycle
model.add(Dense(num_classes)) # there are 10 classes i.e at output layer
model.add(Activation('softmax'))

In [35]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 512)               512512    
_________________________________________________________________
activation_5 (Activation)    (None, 512)               0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                5130      
_________________________________________________________________
activation_6 (Activation)    (None, 10)                0         
Total params: 517,642
Trainable params: 517,642
Non-trainable params: 0
_________________________________________________________________


In [37]:
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

Train on 5443 samples, validate on 605 samples
Epoch 1/10
5443/5443 [==============================] - 1s 167us/step - loss: 1.4656 - accuracy: 0.5510 - val_loss: 1.3014 - val_accuracy: 0.6182
Epoch 2/10
5443/5443 [==============================] - 1s 154us/step - loss: 1.0333 - accuracy: 0.6597 - val_loss: 1.2538 - val_accuracy: 0.6314
Epoch 3/10
5443/5443 [==============================] - 1s 152us/step - loss: 0.8307 - accuracy: 0.7314 - val_loss: 1.2997 - val_accuracy: 0.6264
Epoch 4/10
5443/5443 [==============================] - 1s 153us/step - loss: 0.6921 - accuracy: 0.7784 - val_loss: 1.3802 - val_accuracy: 0.6231
Epoch 5/10
5443/5443 [==============================] - 1s 171us/step - loss: 0.5740 - accuracy: 0.8214 - val_loss: 1.4172 - val_accuracy: 0.5917
Epoch 6/10
5443/5443 [==============================] - 1s 175us/step - loss: 0.4912 - accuracy: 0.8536 - val_loss: 1.5551 - val_accuracy: 0.6099
Epoch 7/10
5443/5443 [==============================] - 1s 160us/step - loss:

In [38]:
score = model.evaluate(x_test, y_test,
                       batch_size=batch_size, verbose=1)
print('Test accuracy:', score[1])

2592/2592 [==============================] - 0s 23us/step
Test accuracy: 0.6242284178733826


# END OF NOTEBOOK